In [6]:
from requests import get
import pandas as pd
from datetime import datetime
import json


from modules.data_fetcher import download_historical_data

# Fear and Greed fetching

In [11]:

response_fng = get('https://api.alternative.me/fng/?limit=0&format=json').text

fng_df = pd.DataFrame(json.loads(response_fng)['data'])

fng_df.drop(columns=['time_until_update'],inplace=True)
fng_df.rename(columns={'timestamp':'date'},inplace=True)

fng_df['date'] = fng_df['date'].apply(lambda x: datetime.fromtimestamp(int(x)))

fng_df.sort_values('date',inplace=True)
fng_df.set_index('date',inplace=True)
fng_df = fng_df.asfreq('1D').ffill()

print(fng_df.shape)
fng_df.head()

(1852, 2)


,value,value_classification
date,,
2018-02-01,30,Fear
2018-02-02,15,Extreme Fear
2018-02-03,40,Fear
2018-02-04,24,Extreme Fear
2018-02-05,11,Extreme Fear


# History fetching

In [17]:
btc_df = download_historical_data('BTC-USDT','1day').loc[fng_df.index[0].strftime("%Y-%m-%d"):]
btc_df = btc_df.asfreq('1D').ffill()
print(btc_df.shape)
btc_df.head()

(1852, 7)


,Timestamp,Open,Close,High,Low,Amount,Volume
Date,,,,,,,
2018-02-01 01:00:00,1.517443e+09,10233.000000,9289.870332,10449.888887,8753.804313,397.353723,3.757247e+06
2018-02-02 01:00:00,1.517530e+09,9289.870332,8849.999994,9390.283108,7863.304872,600.803420,5.168853e+06
2018-02-03 01:00:00,1.517616e+09,8859.999995,9199.999443,9498.999721,8201.999000,268.980095,2.428066e+06
2018-02-04 01:00:00,1.517702e+09,9107.736677,8117.000002,9409.999999,7800.000000,382.837194,3.277363e+06
2018-02-05 01:00:00,1.517789e+09,8210.697671,6950.001000,8379.999999,6470.000001,943.740074,6.945778e+06


In [18]:
pd.merge_asof(btc_df,fng_df, left_index=True, right_index=True,direction='nearest')

,Timestamp,Open,Close,High,Low,Amount,Volume,value,value_classification
Date,,,,,,,,,
2018-02-01 01:00:00,1.517443e+09,10233.000000,9289.870332,10449.888887,8753.804313,397.353723,3.757247e+06,30,Fear
2018-02-02 01:00:00,1.517530e+09,9289.870332,8849.999994,9390.283108,7863.304872,600.803420,5.168853e+06,15,Extreme Fear
2018-02-03 01:00:00,1.517616e+09,8859.999995,9199.999443,9498.999721,8201.999000,268.980095,2.428066e+06,40,Fear
2018-02-04 01:00:00,1.517702e+09,9107.736677,8117.000002,9409.999999,7800.000000,382.837194,3.277363e+06,24,Extreme Fear
2018-02-05 01:00:00,1.517789e+09,8210.697671,6950.001000,8379.999999,6470.000001,943.740074,6.945778e+06,11,Extreme Fear
...,...,...,...,...,...,...,...,...,...
2023-02-22 01:00:00,1.677024e+09,24448.900000,24182.800000,24474.900000,23580.000000,5762.579082,1.380401e+08,59,Greed
2023-02-23 01:00:00,1.677110e+09,24182.800000,23936.300000,24594.000000,23609.800000,5593.856070,1.347439e+08,56,Greed
2023-02-24 01:00:00,1.677197e+09,23936.300000,23187.000000,24133.200000,22844.000000,5478.770118,1.285076e+08,53,Neutral


# Merging Fear and Greed with Price

In [10]:
# fig = make_subplots(
#     rows=2, cols=1, subplot_titles=("Historical price", "Fear and Greed indicator")
# )

# fig.add_trace(
#     go.Candlestick(
#         name="Historical price",
#         x=btc_df.index,
#         open=btc_df["Open"],
#         high=btc_df["High"],
#         low=btc_df["Low"],
#         close=btc_df["Close"],
#     ),
#     row=1,
#     col=1,
# )

# top = 6
# assert top <= len(
#     pkx
# ), "Error provide a lower top value, there are less peaks detected."

# for peak in pkx[np.argpartition(pky, -top)[-top:]]:
#     fig.add_shape(
#         type="line",
#         x0=btc_df.index[0].strftime("%Y-%m-%d %H:%M:%S"),
#         y0=peak,
#         x1=btc_df.index[-1].strftime("%Y-%m-%d %H:%M:%S"),
#         y1=peak,
#         line={"color": "Black"},
#         xref="x",
#         yref="y",
#         row=1,
#         col=1,
#     )

# fig.add_trace(
#     go.Histogram(
#         name="Volume profile",
#         x=btc_df["Volume"],
#         y=btc_df["Close"],
#         nbinsy=150,
#         orientation="h",
#         histfunc="sum",
#         marker_color="#B0C4DE",
#     ),
#     row=1,
#     col=2,
# )
# fig.update_layout(
#     xaxis_rangeslider_visible=False,
#     showlegend=False,
#     title_text="Comparing volume profile and price",
# )
# fig.show()